In [4]:
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table



In [5]:
# Get Project Root and ObsID from the environment
%matplotlib notebook
os.environ['PROJECT_ROOT'] = '/media/kyle/kyle_phd/Swift-j1858.6-0814/'
os.environ['OBSID'] = '/0865600201'
project_root = os.environ.get('PROJECT_ROOT')
obsid = os.environ.get('OBSID')


In [6]:

# CONFIGURATION


# 1. Observation Start Time (MET)
# Define path to the raw lightcurve
lc_file_path = f"{project_root}/products/{obsid}/pn/lc/pn_source_lc_raw.fits"

# Read the RATE table to get time and rate columns
with fits.open(lc_file_path) as hdul:
    lc_table = Table(hdul[1].data)
# Extract time and rate
time = lc_table['TIME'] # This is in MET
rate = lc_table['RATE']

# Get the observation start time from the header
with fits.open(lc_file_path) as hdul:
    header = hdul[1].header
    tstart = header['TSTART'] # This is the MET start time
    tend = header['TSTOP']

# 2. Define the Time Region to Split
# Use the same dipping intervals identified in Step 5.
# Format: List of relative (start, stop) tuples.
dipping_intervals = [
    (3510, 32430)
    # Add other intervals here if your dipping region is split
]


# LOGIC: GENERATE SAS STRING


def format_sas_string(intervals, t_start):
    """Converts intervals to a single SAS boolean expression string."""
    parts = []
    for start, end in intervals:
        abs_start = t_start + start
        abs_end = t_start + end
        parts.append(f"(TIME IN [{abs_start:.6f}:{abs_end:.6f}])")

    # Join with OR (||) so the filter includes ALL pieces of the dipping region
    if not parts:
        return ""
    if len(parts) == 1:
        return parts[0]
    return "(" + " || ".join(parts) + ")"

# Generate the string
dip_filter_string = format_sas_string(dipping_intervals, tstart)


# OUTPUT


print("-" * 60)
print("COPY THIS INTO scripts/06_extract_flux_resolved_spectra.sh")
print("-" * 60)
print("")
print(f'DIP_TIME_FILTER="{dip_filter_string}"')
print("")
print("-" * 60)

------------------------------------------------------------
COPY THIS INTO scripts/06_extract_flux_resolved_spectra.sh
------------------------------------------------------------

DIP_TIME_FILTER="(TIME IN [701596278.492053:701625198.492053])"

------------------------------------------------------------


In [ ]:
!../scripts/06_extract_flux_resolved_spectra.sh > logs/06_extract_flux_resolved_spectra.log 2>&1

Output: The resulting files (spectra, responses, and grouping) will be saved in: products/[OBSID]/pn/spec/flux_resolved/